### This notebook is to test if a CNN is fast enough at predicting the class of a given numpy array.

In [5]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical

# Paths to the directories
base_dir = 'instruments-aug-np/'
categories = ['clap', 'hihat', 'kick', 'snare']

# Load Data function
def load_data(base_dir, categories, target_length=24384):
    data = []
    labels = []
    
    for idx, category in enumerate(categories):
        category_path = os.path.join(base_dir, category)
        for file in os.listdir(category_path):
            if file.endswith('.npy'):
                file_path = os.path.join(category_path, file)
                npy_file = np.load(file_path)
                data.append(npy_file)
                labels.append(idx)

    # Pad all data to the target_length
    padded_data = []
    for npy_file in data:
        if len(npy_file) < target_length:
            # Pad with zeros to match target_length
            padded_file = np.pad(npy_file, (0, target_length - len(npy_file)), mode='constant')
        else:
            # Truncate if the file is longer than target_length
            padded_file = npy_file[:target_length]
        padded_data.append(padded_file)

    return np.array(padded_data), np.array(labels)

# Load the data and corresponding labels
X, y = load_data(base_dir, categories)

# Normalize the data
X = X / np.max(X)

# One-hot encode the labels
y = to_categorical(y, num_classes=len(categories))

# Train-Validation-Test Split (70-10-20)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# Reshape data for 1D CNN (keeping the input as sequences, no need to reshape into 2D)
X_train = X_train.reshape(-1, 24384, 1)
X_val = X_val.reshape(-1, 24384, 1)
X_test = X_test.reshape(-1, 24384, 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

# Define the 1D CNN model in TensorFlow using Input layer
model = Sequential([
    Input(shape=(24384, 1)),  # Explicitly define the input shape here
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/sahil/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sahil

AttributeError: _ARRAY_API not found

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [6]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Paths to the directories
base_dir = 'instruments-aug-np/'
categories = ['clap', 'hihat', 'kick', 'snare']

# Custom Dataset class for PyTorch
class InstrumentDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return sample, label

# Load Data function
def load_data(base_dir, categories, target_length=24384):
    data = []
    labels = []
    
    for idx, category in enumerate(categories):
        category_path = os.path.join(base_dir, category)
        for file in os.listdir(category_path):
            if file.endswith('.npy'):
                file_path = os.path.join(category_path, file)
                npy_file = np.load(file_path)
                data.append(npy_file)
                labels.append(idx)

    # Pad all data to the target_length
    padded_data = []
    for npy_file in data:
        if len(npy_file) < target_length:
            padded_file = np.pad(npy_file, (0, target_length - len(npy_file)), mode='constant')
        else:
            padded_file = npy_file[:target_length]
        padded_data.append(padded_file)

    return np.array(padded_data), np.array(labels)

# Load the data and corresponding labels
X, y = load_data(base_dir, categories)

# Normalize the data
X = X / np.max(X)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Train-Validation-Test Split (70-10-20)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# Reshape data for 1D CNN (keeping the input as sequences, no need to reshape into 2D)
X_train = X_train.reshape(-1, 24384, 1)
X_val = X_val.reshape(-1, 24384, 1)
X_test = X_test.reshape(-1, 24384, 1)

# Create Dataset and DataLoader
train_dataset = InstrumentDataset(X_train, y_train)
val_dataset = InstrumentDataset(X_val, y_val)
test_dataset = InstrumentDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the 1D CNN model in PyTorch
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 3030, 128)  # Adjust based on the input size after convolution and pooling
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 4)  # 4 output classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3030)  # Flatten the output for the fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.permute(0, 2, 1), torch.argmax(labels, dim=1)  # Permute for Conv1D

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loss
        model.eval()
        val_loss = 0.0
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.permute(0, 2, 1), torch.argmax(labels, dim=1)  # Permute for Conv1D
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Test the model
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.permute(0, 2, 1), torch.argmax(labels, dim=1)  # Permute for Conv1D
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

# Evaluate the model on the test set
test_model(model, test_loader)


RuntimeError: shape '[-1, 387840]' is invalid for input of size 12476416

In [23]:
X, y = load_data(base_dir, categories)

In [50]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 9.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 8.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 7.4 MB/s eta 0:00:00


In [2]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

# Paths to the directories
base_dir = 'instruments-aug-np/'
categories = ['clap', 'hihat', 'kick', 'snare']

# Load Data function
def load_data(base_dir, categories, target_length=24384):
    data = []
    labels = []
    
    for idx, category in enumerate(categories):
        category_path = os.path.join(base_dir, category)
        for file in os.listdir(category_path):
            if file.endswith('.npy'):
                file_path = os.path.join(category_path, file)
                npy_file = np.load(file_path)
                data.append(npy_file)
                labels.append(idx)

    # Pad all data to the target_length
    padded_data = []
    for npy_file in data:
        if len(npy_file) < target_length:
            # Pad with zeros to match target_length
            padded_file = np.pad(npy_file, (0, target_length - len(npy_file)), mode='constant')
        else:
            # Truncate if the file is longer than target_length
            padded_file = npy_file[:target_length]
        padded_data.append(padded_file)

    return np.array(padded_data), np.array(labels)

# Load the data and corresponding labels
X, y = load_data(base_dir, categories)

# Normalize the data
X = X / np.max(X)

# One-hot encode the labels
y = to_categorical(y, num_classes=len(categories))

# Train-Validation-Test Split (70-10-20)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# Reshape data for 1D CNN (keeping the input as sequences, no need to reshape into 2D)
X_train = X_train.reshape(-1, 24384, 1)
X_val = X_val.reshape(-1, 24384, 1)
X_test = X_test.reshape(-1, 24384, 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

# Define the 1D CNN model in TensorFlow
model = Sequential()

# Add 1D Conv layer
model.add(Conv1D(24384, kernel_size=3, activation='relu', input_shape=(24384, 1)))
model.add(MaxPooling1D(pool_size=2))

# # Add more 1D Conv layers
model.add(Conv1D(12192, kernel_size=3, activation='relu', input_shape=(12192, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(6096, kernel_size=3, activation='relu', input_shape=(6096, 1)))
model.add(MaxPooling1D(pool_size=6))

model.add(Conv1D(1016, kernel_size=3, activation='relu', input_shape=(1016, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(508, kernel_size=3, activation='relu', input_shape=(508, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(254, kernel_size=3, activation='relu', input_shape=(254, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(127, kernel_size=3, activation='relu', input_shape=(127, 1)))
model.add(MaxPooling1D(pool_size=2))

# model.add(Conv1D(128, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Fully connected Dense layers
model.add(Dense(127, activation='relu'))
model.add(Dropout(0.5))

# Output layer for the 4 classes
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/sahil/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sahil

AttributeError: _ARRAY_API not found

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception